In [1]:
#We load in all necessary libraries.
from tqdm import tqdm
import pandas as pd
import json
from pandas import json_normalize
import re
from collections import Counter
from datetime import datetime
import numpy as np

## Combining per-Hour CSV Files

In [2]:
#We create a list of the existing newspapers in the data.
newspapers = ["gooieneemlander", "haarlemsdagblad", "ijmuidercourant", "leidschdagblad", "noordhollandsdagblad"]

#We create an empty list.
df = []

#We create an object equal to zero.
length = 0

#We loop over all the newspapers.
for n in tqdm(newspapers):
    #We store each newspaper per hour file as a seperate object, by their respective names.
    globals()[n] = pd.read_csv(f"{n}.nl-perhour.csv")
    #We add to it a column with the name of the newspaper.
    globals()[n]["Newspaper"] = n.title()
    
    #We sum the length of the current data-object with the zero object.
    length += len(globals()[n])
    
    #We add the newspaper per-hour objects to a list.
    df.append(globals()[n])

#We stack the list of data objects together.
df = pd.concat(df, ignore_index = True)

#We check if the length of the seperate objects combined equals the length of the definitive dataframe.
print(f"Length of df equals sum of seperate newspaper dataframes = {len(df) == length}.")

100%|██████████| 5/5 [01:22<00:00, 16.47s/it]


Length of df equals sum of seperate newspaper dataframes = True.


In [3]:
#We convert the date to a datetime object.
df["timestamp"] = pd.to_datetime(df.timestamp, format="%Y-%m-%d %H:%M:%S")

In [4]:
#We use a regular expression to extract the landing page.
df["Landing page"] = df["Bestemmingspagina"].apply(lambda x: re.search(r"\/cnt\/dmf(\d+)_\d+", x.lower()).group(0))

In [5]:
#We quickly sort our dataset on the length of the landing page, to see if the streamlining went correctly,
#Thus if we do not have very short or very long landing page ids.
df.sort_values(by="Landing page", key=lambda x: x.str.len())

,Bestemmingspagina,timestamp,Sessies,Newspaper,Landing page
74127110,/cnt/dmf20191229_491,2021-06-06 22:00:00,0,Noordhollandsdagblad,/cnt/dmf20191229_491
90327868,/cnt/dmf20200812_110,2021-05-06 09:00:00,0,Noordhollandsdagblad,/cnt/dmf20200812_110
90327867,/cnt/dmf20200812_110,2021-05-06 08:00:00,0,Noordhollandsdagblad,/cnt/dmf20200812_110
90327866,/cnt/dmf20200812_110,2021-05-06 07:00:00,0,Noordhollandsdagblad,/cnt/dmf20200812_110
90327865,/cnt/dmf20200812_110,2021-05-06 06:00:00,0,Noordhollandsdagblad,/cnt/dmf20200812_110
...,...,...,...,...,...
44140140,/cnt/dmf20210415_52244494,2021-06-30 12:00:00,0,Leidschdagblad,/cnt/dmf20210415_52244494
44140139,/cnt/dmf20210415_52244494,2021-06-30 11:00:00,0,Leidschdagblad,/cnt/dmf20210415_52244494
44140138,/cnt/dmf20210415_52244494,2021-06-30 10:00:00,0,Leidschdagblad,/cnt/dmf20210415_52244494
44140164,/cnt/dmf20210415_52244494,2021-07-01 12:00:00,0,Leidschdagblad,/cnt/dmf20210415_52244494


In [6]:
#We drop the original column.
df.drop("Bestemmingspagina", axis=1, inplace = True)

In [7]:
#We rename a few columns.
df.rename(columns = {"timestamp": "Timestamp date", "Sessies": "Views"}, inplace = True)

In [8]:
#We inspect the data.
df.head()

,Timestamp date,Views,Newspaper,Landing page
0,2021-05-06 12:00:00,1,Gooieneemlander,/cnt/dmf20180830_13090784
1,2021-06-12 22:00:00,1,Gooieneemlander,/cnt/dmf20180830_15395191
2,2021-04-13 17:00:00,1,Gooieneemlander,/cnt/dmf20180830_21070680
3,2021-04-13 18:00:00,0,Gooieneemlander,/cnt/dmf20180830_21070680
4,2021-04-13 19:00:00,0,Gooieneemlander,/cnt/dmf20180830_21070680


## Combining JSON Dictionaries

In [9]:
#We create a list of newspapers again.
newspapers = ["gooieneemlander", "haarlemsdagblad", "ijmuidercourant", "leidschdagblad", "noordhollandsdagblad"]

#We create a new empty list.
df2 = []

#We loop over the newspapers.
for n in tqdm(newspapers):
    #Open their respective json files.
    with open(f"{n}.json", "r") as f:
        #And we add the data line per line.
        for line in f:
            line = json.loads(line)
            df2.append(line)

#We normalize the data to get it in a tabular format.
df2 = json_normalize(df2)

100%|██████████| 5/5 [00:14<00:00,  2.95s/it]


In [10]:
#We inspect the data.
df2.head()

,url,meta,meta-script.interactiefPrefix,meta-script.brooklynapiurl,meta-script.readLaterServiceUrl,meta-script.froomleServiceUrl,meta-script.access_loginshown,meta-script.article_author,meta-script.article_bodycharcount,meta-script.article_positive,...,meta-script.cdn_static,meta-script.article_has_inline_media,meta-script.urlscheme,meta-script.page_features,meta-script.environment,meta-script.metered,meta-script.visitorIsValidBot,meta-script.page_zipcode,meta-script.page_cityname,meta-script.article_categorylist
0,https://www.gooieneemlander.nl/cnt/dmf20210612...,"[{'charset': 'utf-8'}, {'name': 'cXenseParse:m...",http,https://brooklynapi.gooieneemlander.nl,/api/readlater/,/api/froomle/,False,ANP,4888.0,True,...,cdn-kiosk-api.telegraaf.nl,True,https,[],production,False,False,NaN,NaN,NaN
1,https://www.gooieneemlander.nl/cnt/dmf20210611...,"[{'charset': 'utf-8'}, {'name': 'cXenseParse:m...",http,https://brooklynapi.gooieneemlander.nl,/api/readlater/,/api/froomle/,False,Internetredactie,839.0,True,...,cdn-kiosk-api.telegraaf.nl,True,https,[],production,False,False,"1111,1111","Dummy regio,Dummy regio",NaN
2,https://www.gooieneemlander.nl/cnt/dmf20210609...,"[{'charset': 'utf-8'}, {'name': 'cXenseParse:m...",http,https://brooklynapi.gooieneemlander.nl,/api/readlater/,/api/froomle/,False,Susanne van Velzen,3403.0,True,...,cdn-kiosk-api.telegraaf.nl,True,https,[],production,False,False,"1111,1111","Dummy regio,Dummy regio",NaN
3,https://www.gooieneemlander.nl/cnt/dmf20210609...,"[{'charset': 'utf-8'}, {'name': 'cXenseParse:m...",http,https://brooklynapi.gooieneemlander.nl,/api/readlater/,/api/froomle/,False,Internetredactie,1807.0,True,...,cdn-kiosk-api.telegraaf.nl,True,https,[],production,False,False,"1111,1111","Dummy regio,Dummy regio",NaN
4,https://www.gooieneemlander.nl/cnt/dmf20210608...,"[{'charset': 'utf-8'}, {'name': 'cXenseParse:m...",http,https://brooklynapi.gooieneemlander.nl,/api/readlater/,/api/froomle/,False,Leander Mascini,2146.0,True,...,cdn-kiosk-api.telegraaf.nl,True,https,[],production,False,False,"1111,1111","Dummy regio,Dummy regio",NaN


In [11]:
#We inspect the columns.
df2.columns

Index(['url', 'meta', 'meta-script.interactiefPrefix',
       'meta-script.brooklynapiurl', 'meta-script.readLaterServiceUrl',
       'meta-script.froomleServiceUrl', 'meta-script.access_loginshown',
       'meta-script.article_author', 'meta-script.article_bodycharcount',
       'meta-script.article_positive', 'meta-script.article_cimpage',
       'meta-script.article_cimpage_free', 'meta-script.article_cityname',
       'meta-script.article_contenttype', 'meta-script.article_dossier',
       'meta-script.article_id', 'meta-script.article_introcharcount',
       'meta-script.article_sectionid', 'meta-script.article_section',
       'meta-script.article_maintag', 'meta-script.article_paper',
       'meta-script.article_publicationdate',
       'meta-script.article_publicationtime',
       'meta-script.article_ispluslayout', 'meta-script.article_source',
       'meta-script.article_sourceoforigin', 'meta-script.article_taglist',
       'meta-script.article_title', 'meta-script.article_t

In [18]:
#We apply our earlier defined regular expression to clean landing page urls.
df2["Landing page"] = df2["url"].apply(lambda x: re.search(r"\/cnt\/dmf(\d+)_\d+", x.lower()).group(0))

In [19]:
#We quickly inspect it in the same manner as before.
df2.sort_values(by="Landing page", key=lambda x: x.str.len())[["Landing page"]]

,Landing page
12448,/cnt/dmf20191229_491
120634,/cnt/dmf20191229_491
60606,/cnt/dmf20200812_110
107952,/cnt/dmf20200812_110
35769,/cnt/dmf20200812_110
...,...
73599,/cnt/dmf20190604_32241827
73600,/cnt/dmf20210331_18308698
73602,/cnt/dmf20180926_49230990
73629,/cnt/dmf20200803_19491311


In [20]:
#We extract the newspaper from the url as well (by simply looking at where www. starts and .nl ends).
df2["Newspaper"] = df2["url"].apply(lambda x:x[x.find("www.") + len("www."):x.find(".nl")].title())

In [21]:
#We count the presence of the newspaper (and indirectly check if missing values have occured).
Counter(df2.Newspaper)

Counter({'Gooieneemlander': 24438,
         'Haarlemsdagblad': 26273,
         'Ijmuidercourant': 16433,
         'Leidschdagblad': 30476,
         'Noordhollandsdagblad': 111875})

In [22]:
#We only keep the columns that we need.
df2 = df2[["meta-script.article_publicationdate", "meta-script.article_publicationtime", "Landing page", "Newspaper"]]

In [23]:
#We rename some of them.
df2.rename(columns = {"meta-script.article_publicationdate": "Publication date",
                      "meta-script.article_publicationtime": "Publication time"}, inplace = True)

In [24]:
#We check if there are publication dates and times missing.
print(Counter(df2["Publication date"].isna()))
print(Counter(df2["Publication time"].isna()))

Counter({False: 208348, True: 1147})
Counter({False: 208348, True: 1147})


In [25]:
#We create an empty list.
Publication_date = []

#We loop over the date and times.
for d, t in tqdm(zip(df2["Publication date"], df2["Publication time"])):
    #We check if one of the two is missing.
    if pd.isnull(d) == True | pd.isnull(t) == True:
        #If so we simply append nothing.
        Publication_date.append("")
    else:
        #If not we remove the minutes of the time object.
        item = d + t[:-2] + "00"
        #We streamline it.
        item = datetime.strptime(item, "%Y%m%d%H:%M")
        #And append it to a list.
        Publication_date.append(item)

209495it [00:03, 57003.63it/s]


In [26]:
#We drop the original columns.
df2 = df2.drop(columns = ["Publication date", "Publication time"])

#And add our list as a new column.
df2["Publication date"] = Publication_date

/tmp/ipykernel_4085/696928151.py:5: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  df2["Publication date"] = Publication_date


In [27]:
#We write this dataframe away.
df2.to_csv("Publication dates.csv", index=False)

## Merging the JSON Files and CSV Files to Remove Old Publications

In [28]:
#We merge the cleaned combined json files with the clean combined csv per-hour files.
df3 = df.merge(df2, how="left", on=["Newspaper", "Landing page"])

In [29]:
#We inspect it once again (better safe than sorry).
df3.head()

,Timestamp date,Views,Newspaper,Landing page,Publication date
0,2021-05-06 12:00:00,1,Gooieneemlander,/cnt/dmf20180830_13090784,2018-08-29 16:00:00
1,2021-06-12 22:00:00,1,Gooieneemlander,/cnt/dmf20180830_15395191,2018-08-29 20:00:00
2,2021-04-13 17:00:00,1,Gooieneemlander,/cnt/dmf20180830_21070680,2018-08-30 14:00:00
3,2021-04-13 18:00:00,0,Gooieneemlander,/cnt/dmf20180830_21070680,2018-08-30 14:00:00
4,2021-04-13 19:00:00,0,Gooieneemlander,/cnt/dmf20180830_21070680,2018-08-30 14:00:00


In [30]:
#We check when data collection started.
df3.groupby(["Newspaper"])["Timestamp date"].min()

Newspaper
Gooieneemlander        2021-04-12
Haarlemsdagblad        2021-04-12
Ijmuidercourant        2021-04-12
Leidschdagblad         2021-04-12
Noordhollandsdagblad   2021-04-12
Name: Timestamp date, dtype: datetime64[ns]

In [31]:
#We subset the data accordingly to make sure that we have data on the full trajectory of each article. 
#We thus exclude articles that are published before the data collection started.
#To be sure we also exclude timestamps that occured before the publication date.
df3 = df3[(df3["Publication date"] >= "2021-04-12 00:00:00") & (df3["Timestamp date"] >= df3["Publication date"])]

In [32]:
#We inspect the data.
df3.head()

,Timestamp date,Views,Newspaper,Landing page,Publication date
5326326,2021-05-27 16:00:00,1,Gooieneemlander,/cnt/dmf20190705_81476516,2021-05-26 19:00:00
5326327,2021-05-27 17:00:00,0,Gooieneemlander,/cnt/dmf20190705_81476516,2021-05-26 19:00:00
5326328,2021-05-27 18:00:00,0,Gooieneemlander,/cnt/dmf20190705_81476516,2021-05-26 19:00:00
5326329,2021-05-27 19:00:00,0,Gooieneemlander,/cnt/dmf20190705_81476516,2021-05-26 19:00:00
5326330,2021-05-27 20:00:00,0,Gooieneemlander,/cnt/dmf20190705_81476516,2021-05-26 19:00:00


In [33]:
#We create a function to add a row.
def add_row(x):
    #Namely we check for publications where the first timestamp is not equal to the publication date.
    if x["Timestamp date"].min() != x["Publication date"].iloc[0]:
        #If so we add a new custom row to the top of the publication.
        #This row will have the same information as the other rows, having zero views and having a timestamp equal to the publication date.
        #This is done so that the gap between the publication date and the first timestamp can be filled up later on.
        return pd.concat([pd.DataFrame({"Landing page": x["Landing page"].iloc[0],
                                        "Views": 0,
                                        "Newspaper": x["Newspaper"].iloc[0],
                                        "Publication date": x["Publication date"].iloc[0],
                                        "Timestamp date": x["Publication date"].iloc[0]
                                       }, index=[0]), x]).reset_index(drop=True)
    return x

In [34]:
#We apply the function per publication.
df3 = df3.groupby(["Newspaper", "Landing page"]).apply(add_row).reset_index(drop=True)

In [35]:
#We fill up the spaces between the timestamps with rows of 1 hour.
#.sum() is used so that views get a value of zero (the whole reason that the timestamp was missing was because there were zero views).
#In addition we keep the original publication date.
df3 = df3.groupby(["Newspaper", "Landing page"])\
         .apply(lambda x: x.set_index("Timestamp date").resample("1H").sum()\
         .assign(**{"Publication date": x["Publication date"].iloc[0]})).reset_index()

In [36]:
#We inspect the data.
df3.head()

,Newspaper,Landing page,Timestamp date,Views,Publication date
0,Gooieneemlander,/cnt/dmf20190705_81476516,2021-05-26 19:00:00,0,2021-05-26 19:00:00
1,Gooieneemlander,/cnt/dmf20190705_81476516,2021-05-26 20:00:00,0,2021-05-26 19:00:00
2,Gooieneemlander,/cnt/dmf20190705_81476516,2021-05-26 21:00:00,0,2021-05-26 19:00:00
3,Gooieneemlander,/cnt/dmf20190705_81476516,2021-05-26 22:00:00,0,2021-05-26 19:00:00
4,Gooieneemlander,/cnt/dmf20190705_81476516,2021-05-26 23:00:00,0,2021-05-26 19:00:00


In [37]:
#As a first check we check if the first timestamp now indeed equals the publication date.
Counter(df3.groupby(["Newspaper", "Landing page"])\
        .apply(lambda x: (x["Publication date"].iloc[0] == x["Timestamp date"].iloc[0])))

Counter({True: 46550})

In [38]:
#In addition we check if now the difference between each row per publication is 1 hour.
Counter(df3.groupby(["Newspaper", "Landing page"])["Timestamp date"].diff())

Counter({NaT: 46550, Timedelta('0 days 01:00:00'): 17737802})

In [39]:
#We quickly inspect the length.
len(df3)

17784352

In [40]:
#We write it away to a csv file.
df3.to_csv("New Publications.csv", index=False)